This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [ ]:
import tensorflow as tf
import numpy as np
import time
import importlib

In [ ]:
import sys
sys.path.append('../py')
import cavitylearn.data
import cavitylearn.catalonet0

## Loading Data

In [ ]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=8, boxshape=[30, 30, 30])
with open('/var/cavitylearn/dataset-test/labels.txt', 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, '/var/cavitylearn/dataset-test', dataconfig)

In [ ]:
print('Examples available in dataset: ', trainset.N)

## Building the model

In [ ]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [ ]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [ ]:
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(labels_placholder * tf.log(logits), reduction_indices=[1]))
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
#  logits, labels_placholder, name='cross_entropy_per_example')

In [ ]:
loss = cavitylearn.catalonet0.loss(logits, labels_placholder)

In [ ]:
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training the Model

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
BATCHSIZE = 100
NUM_BATCHES = 10

In [ ]:
trainset.rewind_batches()
start_time = time.time()
for i in range(NUM_BATCHES):
    labels, boxes = trainset.next_batch(BATCHSIZE)
    if len(labels) == 0:
        break
    sess.run(train_step, feed_dict={boxes_placeholder: boxes, labels_placholder: labels})
    print(';', str(sess.run(accuracy, feed_dict={boxes_placeholder: boxes, labels_placholder: labels})), end='')
end_time = time.time()

In [ ]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

In [ ]:
importlib.reload(cavitylearn.catalonet0)
tf.reset_default_graph()